In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from config import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

In [ ]:
#perform weather check on each city using a series of successive API calls
#include a print log of each city as it's being processed with city number and city name

base_url = "api.openweathermap.org/data/2.5/weather?q={city name}&appid={api_key}"

#create settings dictionary (learned code from JKora)
settings = {"units": "imperial", "appid": api_key}

#create data frame to store data from api calls
WeatherPy_df = pd.DataFrame(columns = ["City", "Country","Lat", "Lng", "Date", 
                                       "Cloudiness","Max Temp", "Wind Speed",
                                      "Humidity"])
        

In [ ]:
#perform weather check on each city using successive API calls with print log
#learned code from JKora

count = 1 #to count number of calls to API
err = 0 #to record how many cities come back with no data
sets = 0 #counts number of call sets
index = 0 #index for datatframe 
t0 = time.time() #to get current time

print(f'Beginning Data Retrieval \n ..............')
for city in cities:
    try: 
        city1 = city.replace(" ", "%20") #this removes space in city name so URL isn't broken
        print(f'Processing Record {count} of Set {sets} | {city}')
        city_stats = base_url.get_current(city, **settings)
        WeatherPy_df.set_value(index, "City", city_stats["name"])
        WeatherPy_df.set_value(index, "Country", city_stats["sys.country"])
        WeatherPy_df.set_value(index, "Lat", city_stats["coord.lat"])
        WeatherPy_df.set_value(index, "Lng", city_stats["coord.lng"])
        WeatherPy_df.set_value(index, "Date", city_stats["dt"])
        WeatherPy_df.set_value(index, "Cloudiness", city_stats["clouds.all"])
        WeatherPy_df.set_value(index, "Max Temp", city_stats["main.temp_max"])
        WeatherPy_df.set_value(index, "Wind Speed", city_stats["wind.speed"])
        WeatherPy_df.set_value(index, "Humidity", city_stats["main.humidity"])
        index = index+1
    except Exception as e:
        print(e)
        err = err+1
    count = count+1
    if count == 51:
        t1 = time.time()
        count = 1
        sets = sets+1
        time.sleep(60-(t1-t0))
        t0=time.time()
        
print("------------------------")
print("Data Retrieval Complete")
print("------------------------")
print(f'Could not get data for {err} cities')
        